In [1]:
# 🔧 MoE Model Conversion - CORRECTED Configuration
# 
# ✅ FIXED ISSUES after reading actual tiny-moe codebase:
# 1. vocab_size: 50256 (was 50257) - from config.py
# 2. d_hidden: 384 confirmed (d_model // 2 for TOTAL params) - from layer.py comment
# 3. bos/eos token IDs: 50255 (vocab_size - 1)
# 4. All parameters now match actual MoE training configuration

# First, let's examine the MoE checkpoint structure
import torch
import json

# Load an MoE checkpoint to understand structure
def examine_moe_checkpoint(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location="cpu")
    
    if "model_state_dict" in checkpoint:
        state_dict = checkpoint["model_state_dict"]
    else:
        state_dict = checkpoint
    
    print(f"MoE checkpoint keys from {checkpoint_path}:")
    for k in sorted(state_dict.keys()):
        print(f"  {k}: {state_dict[k].shape}")
    
    return state_dict

# Examine a couple of MoE checkpoints
print("Examining MoE checkpoint structure:")
moe_state = examine_moe_checkpoint("moe-activated/best_val_loss_moe_step_6000.pt")

Examining MoE checkpoint structure:
MoE checkpoint keys from moe-activated/best_val_loss_moe_step_6000.pt:
  layers.0.attention.c_attn.weight: torch.Size([2304, 768])
  layers.0.attention.c_proj.weight: torch.Size([768, 768])
  layers.0.attn_norm.w: torch.Size([768])
  layers.0.ffn.experts.0.out.bias: torch.Size([768])
  layers.0.ffn.experts.0.out.weight: torch.Size([768, 1536])
  layers.0.ffn.experts.0.w_1.bias: torch.Size([1536])
  layers.0.ffn.experts.0.w_1.weight: torch.Size([1536, 768])
  layers.0.ffn.experts.0.w_2.bias: torch.Size([1536])
  layers.0.ffn.experts.0.w_2.weight: torch.Size([1536, 768])
  layers.0.ffn.experts.1.out.bias: torch.Size([768])
  layers.0.ffn.experts.1.out.weight: torch.Size([768, 1536])
  layers.0.ffn.experts.1.w_1.bias: torch.Size([1536])
  layers.0.ffn.experts.1.w_1.weight: torch.Size([1536, 768])
  layers.0.ffn.experts.1.w_2.bias: torch.Size([1536])
  layers.0.ffn.experts.1.w_2.weight: torch.Size([1536, 768])
  layers.0.ffn.experts.2.out.bias: torch.Siz

In [2]:
# Create config for MoE model - ACTIVATED PARAMETERS VERSION
# From layer.py: d_hidden = d_model // 2 for "total params", d_hidden = d_model * 2 for "active params"
# User wants ACTIVATED params with d_hidden = 1536, where d_model = 768, so 1536 = 768 * 2 ✓

def create_moe_activated_config():
    """
    Create a GPT2-style config for the MoE model - ACTIVATED PARAMETERS version.
    
    Based on ACTUAL tiny-moe codebase analysis and conversion:
    - d_model = 768 (from config.py)
    - d_hidden = 1536 (d_model * 2 for ACTIVATED params - from layer.py comment!)
    - n_experts = 8, top_k = 2 (from config.py)
    - vocab_size = 50257 (actual trained model)
    - wandb_project = 'moe-active' (from config.py)
    
    Key insight from layer.py line 28:
    # for matching total params just do d_hidden = d_model // 2 & d_hidden = d_model * 2 for matching active params
    """
    config_dict = {
        "model_type": "gpt2",
        "vocab_size": 50257,  # ✅ CORRECTED: actual trained model has 50257 (standard GPT-2)
        "n_positions": 1024,  # max_seq_len from MoE config
        "n_embd": 768,        # d_model from MoE config
        "n_layer": 5,         # ✅ CRITICAL: MoE has exactly 5 layers (0-4)
        "n_head": 12,         # n_heads from MoE config  
        "n_inner": 1536,      # ✅ CRITICAL: d_hidden = 1536 for ACTIVATED params (d_model * 2)
        "activation_function": "gelu_new",
        "resid_pdrop": 0.1,   # dropout from MoE config
        "embd_pdrop": 0.1,    # dropout from MoE config
        "attn_pdrop": 0.1,    # attn_dropout from MoE config
        "layer_norm_epsilon": 1e-06,  # norm_eps from MoE config
        "initializer_range": 0.02,
        "bos_token_id": 50256,  # vocab_size - 1 = 50257 - 1 = 50256
        "eos_token_id": 50256,  # vocab_size - 1 = 50257 - 1 = 50256
        "architectures": ["GPT2LMHeadModel"],
        "task_specific_params": {
            "text-generation": {
                "do_sample": True,
                "max_length": 50
            }
        }
    }
    
    # Save config to both locations
    import os
    os.makedirs("moe-activated-converted", exist_ok=True)
    
    # Save to root (for local testing)
    with open("config.json", "w") as f:
        json.dump(config_dict, f, indent=2)
    
    # Save to converted directory (for HuggingFace upload)
    with open("moe-activated-converted/config.json", "w") as f:
        json.dump(config_dict, f, indent=2)
    
    return config_dict

# Create the ACTIVATED config
config = create_moe_activated_config()
print("✅ Created MoE ACTIVATED PARAMETERS config:")
print(f"  - vocab_size: {config['vocab_size']} (matches actual trained model)")
print(f"  - n_layer: {config['n_layer']} (exactly 5 layers: 0-4)")
print(f"  - n_inner: {config['n_inner']} (d_hidden for ACTIVATED params = d_model * 2)")
print(f"  - bos/eos_token_id: {config['bos_token_id']} (vocab_size - 1)")
print("✅ Config saved to both root and moe-activated-converted/ directories")


✅ Created MoE ACTIVATED PARAMETERS config:
  - vocab_size: 50257 (matches actual trained model)
  - n_layer: 5 (exactly 5 layers: 0-4)
  - n_inner: 1536 (d_hidden for ACTIVATED params = d_model * 2)
  - bos/eos_token_id: 50256 (vocab_size - 1)
✅ Config saved to both root and moe-activated-converted/ directories


In [3]:
def convert_moe_activated_checkpoint_to_hf(checkpoint_path, output_path):
    """
    Convert MoE checkpoint to HuggingFace GPT2 format - ACTIVATED PARAMETERS version.
    
    MoE Architecture (from codebase analysis):
    - Each expert is SwiGLUFFN with w_1, w_2, out layers
    - d_hidden = 1536 (for ACTIVATED params = d_model * 2), d_model = 768
    - 8 experts, top_k = 2
    
    SwiGLU -> GPT2 MLP conversion strategy for ACTIVATED params:
    - Combine/concatenate expert weights to create larger MLP
    - w_1 weights -> c_fc (1536 hidden dim)
    - out weights -> c_proj
    """
    # Load MoE checkpoint
    checkpoint = torch.load(checkpoint_path, map_location="cpu")
    
    if "model_state_dict" in checkpoint:
        state_dict = checkpoint["model_state_dict"]
    else:
        state_dict = checkpoint
    
    # Map MoE model's keys to GPT2 keys
    hf_state_dict = {}
    
    # Collect expert weights by layer for averaging
    expert_weights = {}  # layer_num -> weight_type -> param_type -> expert_id -> tensor
    
    for key, value in state_dict.items():
        # Token embeddings - use actual trained vocab size
        if "tok_embedding.weight" in key:
            hf_state_dict["transformer.wte.weight"] = value
            print(f"✅ Token embeddings: {value.shape}")
            
        # Output layer - use actual trained vocab size
        elif "output.weight" in key:
            hf_state_dict["lm_head.weight"] = value
            print(f"✅ Output weights: {value.shape}")
            
        # Output bias - Skip for standard GPT-2 compatibility
        elif "output.bias" in key:
            # Standard GPT-2 doesn't use lm_head.bias, so skip this
            print(f"⚠️  Skipping output.bias (standard GPT-2 doesn't use lm_head bias)")
            continue
            
        # Final layer norm (same as dense)
        elif "norm.w" in key and "layers" not in key:
            hf_state_dict["transformer.ln_f.weight"] = value
            
        # Transformer layers
        elif "layers" in key:
            parts = key.split(".")
            layer_num = int(parts[1])
            
            # Attention weights (same as dense conversion)
            if "attention.c_attn.weight" in key:
                hf_state_dict[f"transformer.h.{layer_num}.attn.c_attn.weight"] = value.T
            elif "attention.c_proj.weight" in key:
                hf_state_dict[f"transformer.h.{layer_num}.attn.c_proj.weight"] = value.T
                
            # Layer norms (same as dense)
            elif "attn_norm.w" in key:
                hf_state_dict[f"transformer.h.{layer_num}.ln_1.weight"] = value
            elif "ffn_norm.w" in key:
                hf_state_dict[f"transformer.h.{layer_num}.ln_2.weight"] = value
                
            # MoE expert weights - collect for averaging
            elif "ffn.experts" in key:
                # Parse: layers.{layer_num}.ffn.experts.{expert_id}.{weight_type}.{param_type}
                expert_id = int(parts[4])  # expert number (0-7)
                weight_type = parts[5]     # w_1, w_2, or out
                param_type = parts[6]      # weight or bias
                
                # Initialize nested dict structure
                if layer_num not in expert_weights:
                    expert_weights[layer_num] = {}
                if weight_type not in expert_weights[layer_num]:
                    expert_weights[layer_num][weight_type] = {}
                if param_type not in expert_weights[layer_num][weight_type]:
                    expert_weights[layer_num][weight_type][param_type] = {}
                    
                expert_weights[layer_num][weight_type][param_type][expert_id] = value
                
            # Skip router weights (not needed for standard GPT2)
            elif "ffn.router" in key:
                continue  # Silent skip
    
    print(f"Found expert weights for {len(expert_weights)} layers")
    
    # Convert SwiGLU experts to standard GPT2 MLP for each layer
    for layer_num in range(5):  # 5 layers
        if layer_num in expert_weights:
            layer_experts = expert_weights[layer_num]
            
            # SwiGLU has: out(w_1(x) * silu(w_2(x)))
            # Convert to GPT2 MLP: c_proj(gelu(c_fc(x)))
            
            # For ACTIVATED params: Each expert has d_hidden=1536, so average across experts
            # Strategy: Average w_1 weights across experts for c_fc (already 1536 dim)
            if "w_1" in layer_experts and "weight" in layer_experts["w_1"]:
                w1_weights = torch.stack(list(layer_experts["w_1"]["weight"].values()))
                avg_w1_weight = w1_weights.mean(dim=0)  # Shape: (1536, 768) for activated
                # Transpose for GPT2 format: (768, 1536)
                hf_state_dict[f"transformer.h.{layer_num}.mlp.c_fc.weight"] = avg_w1_weight.T
                
            if "w_1" in layer_experts and "bias" in layer_experts["w_1"]:
                w1_biases = torch.stack(list(layer_experts["w_1"]["bias"].values()))
                avg_w1_bias = w1_biases.mean(dim=0)  # Shape: (1536,) for activated
                hf_state_dict[f"transformer.h.{layer_num}.mlp.c_fc.bias"] = avg_w1_bias
            
            # Use out weights for c_proj
            if "out" in layer_experts and "weight" in layer_experts["out"]:
                out_weights = torch.stack(list(layer_experts["out"]["weight"].values()))
                avg_out_weight = out_weights.mean(dim=0)  # Shape: (768, 1536) for activated
                # Transpose for GPT2 format: (1536, 768)
                hf_state_dict[f"transformer.h.{layer_num}.mlp.c_proj.weight"] = avg_out_weight.T
                
            if "out" in layer_experts and "bias" in layer_experts["out"]:
                out_biases = torch.stack(list(layer_experts["out"]["bias"].values()))
                avg_out_bias = out_biases.mean(dim=0)  # Shape: (768,)
                hf_state_dict[f"transformer.h.{layer_num}.mlp.c_proj.bias"] = avg_out_bias
                
        else:
            print(f"Warning: No expert weights found for layer {layer_num}")
    
    # Add missing components that GPT2 expects but MoE model doesn't have
    # 1. Positional embeddings (MoE uses RoPE, initialize to zeros)
    hf_state_dict["transformer.wpe.weight"] = torch.zeros(1024, 768)
    
    # 2. Layer norm bias terms (MoE uses RMSNorm without bias)
    hf_state_dict["transformer.ln_f.bias"] = torch.zeros(768)
    for layer_num in range(5):
        hf_state_dict[f"transformer.h.{layer_num}.ln_1.bias"] = torch.zeros(768)
        hf_state_dict[f"transformer.h.{layer_num}.ln_2.bias"] = torch.zeros(768)
    
    # 3. Attention bias terms (MoE doesn't have these)
    for layer_num in range(5):
        hf_state_dict[f"transformer.h.{layer_num}.attn.c_attn.bias"] = torch.zeros(2304)
        hf_state_dict[f"transformer.h.{layer_num}.attn.c_proj.bias"] = torch.zeros(768)
    
    # Save as pytorch_model.bin
    torch.save(hf_state_dict, output_path)
    print(f"✅ Converted MoE ACTIVATED checkpoint: {checkpoint_path} -> {output_path}")

print("✅ Updated MoE ACTIVATED PARAMETERS conversion function created!")


✅ Updated MoE ACTIVATED PARAMETERS conversion function created!


In [4]:
def test_converted_moe_activated_model(weight_path, test_input):
    """Test the converted MoE ACTIVATED model by loading it and running a simple generation."""
    from transformers import GPT2LMHeadModel, GPT2Config, AutoTokenizer
    import torch

    # Load config from current directory (should be the activated config)
    config = GPT2Config.from_pretrained("./")
    
    print(f"Testing with config: n_inner={config.n_inner} (should be 1536 for activated)")

    # Initialize model
    model = GPT2LMHeadModel(config)

    # Load converted weights
    state_dict = torch.load(weight_path, map_location="cpu")

    # Try loading the state dict
    try:
        missing_keys, unexpected_keys = model.load_state_dict(state_dict, strict=False)
        
        print(f"\nLoading report for {weight_path}:")
        print(f"Missing keys: {len(missing_keys)}")
        if missing_keys:
            print("  ", missing_keys[:5], "..." if len(missing_keys) > 5 else "")
            
        print(f"Unexpected keys: {len(unexpected_keys)}")
        if unexpected_keys:
            print("  ", unexpected_keys[:5], "..." if len(unexpected_keys) > 5 else "")
            
        if len(missing_keys) == 0 and len(unexpected_keys) == 0:
            print("✅ Perfect MoE ACTIVATED conversion!")
            
            # Test the model with a simple generation
            model.eval()
            tokenizer = AutoTokenizer.from_pretrained("gpt2")
            
            # Test generation
            inputs = tokenizer(test_input, return_tensors="pt")
            
            with torch.no_grad():
                outputs = model.generate(inputs["input_ids"], max_length=30, do_sample=True, temperature=0.7)
            
            generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
            print(f"\nTest generation:")
            print(f"Input: {test_input}")
            print(f"Output: {generated_text}")
            
        else:
            print("⚠️  MoE ACTIVATED conversion completed with some mismatches")
            
    except Exception as e:
        print(f"❌ Error during loading: {e}")

print("MoE ACTIVATED test function created!")


MoE ACTIVATED test function created!


In [5]:
# Test the ACTIVATED conversion with a single MoE checkpoint
test_checkpoint = "moe-activated/best_val_loss_moe_step_6900.pt"
test_output = "moe-activated-converted/converted_moe_activated_test.bin"

print(f"🔧 Converting {test_checkpoint} for ACTIVATED parameters...")
convert_moe_activated_checkpoint_to_hf(test_checkpoint, test_output)

print(f"\n🧪 Testing converted ACTIVATED model...")
test_converted_moe_activated_model(test_output, "Once upon a time")


🔧 Converting moe-activated/best_val_loss_moe_step_6900.pt for ACTIVATED parameters...
✅ Token embeddings: torch.Size([50257, 768])
✅ Output weights: torch.Size([50257, 768])
⚠️  Skipping output.bias (standard GPT-2 doesn't use lm_head bias)
Found expert weights for 5 layers
✅ Converted MoE ACTIVATED checkpoint: moe-activated/best_val_loss_moe_step_6900.pt -> moe-activated-converted/converted_moe_activated_test.bin

🧪 Testing converted ACTIVATED model...
Testing with config: n_inner=1536 (should be 1536 for activated)

Loading report for moe-activated-converted/converted_moe_activated_test.bin:
Missing keys: 0
Unexpected keys: 0
✅ Perfect MoE ACTIVATED conversion!


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



Test generation:
Input: Once upon a time
Output: Once upon a time cure accused engaged engaged accused member consequences Bet warning accused accused classmate accused accused accused accused lied betray accused accused accused lied hurt lied betray enemies


In [13]:
# Convert all MoE checkpoints for ACTIVATED parameters
import os
import glob

# Create output directory for converted MoE ACTIVATED models
output_dir = "moe-activated-converted/"
os.makedirs(output_dir, exist_ok=True)

# Find all .pt files in the moe-total/ directory
moe_dir = "moe-activated/"
if os.path.exists(moe_dir):
    pt_files = glob.glob(os.path.join(moe_dir, "*.pt"))
    
    if pt_files:
        print(f"Found {len(pt_files)} MoE checkpoint files in {moe_dir}:")
        for pt_file in pt_files:
            print(f"  - {os.path.basename(pt_file)}")
        
        # Convert each checkpoint to ACTIVATED parameters
        for pt_file in pt_files:
            base_name = os.path.splitext(os.path.basename(pt_file))[0]
            output_name = os.path.join(output_dir, f"{base_name}.bin")
            
            print(f"\n{'='*50}")
            print(f"Converting ACTIVATED: {os.path.basename(pt_file)} -> {output_name}")
            print(f"{'='*50}")
            
            try:
                convert_moe_activated_checkpoint_to_hf(pt_file, output_name)
                print("✅ ACTIVATED conversion successful")
            except Exception as e:
                print(f"❌ ACTIVATED conversion failed: {e}")
    else:
        print(f"No .pt files found in {moe_dir}")
else:
    # print(f"Directory {moe_dir} does not exist")
    # # Fallback to root directory MoE files
    # root_moe_files = glob.glob("best_val_loss_moe_step_*.pt")
    # if root_moe_files:
    #     print(f"Found {len(root_moe_files)} MoE files in root directory")
    #     for pt_file in root_moe_files:
    #         base_name = os.path.splitext(os.path.basename(pt_file))[0]
    #         output_name = os.path.join(output_dir, f"{base_name}.bin")
            
    #         print(f"\n{'='*50}")
    #         print(f"Converting ACTIVATED: {pt_file} -> {output_name}")
    #         print(f"{'='*50}")
            
    #         try:
    #             convert_moe_activated_checkpoint_to_hf(pt_file, output_name)
    #             print("✅ ACTIVATED conversion successful")
    #         except Exception as e:
    #             print(f"❌ ACTIVATED conversion failed: {e}")
    pass

print("\n🎉 All MoE ACTIVATED conversions completed!")


Found 12 MoE checkpoint files in moe-activated/:
  - best_val_loss_moe_step_6000.pt
  - moe_step_9000.pt
  - best_val_loss_moe_step_3600.pt
  - best_val_loss_moe_step_5100.pt
  - best_val_loss_moe_step_8700.pt
  - best_val_loss_moe_step_6900.pt
  - best_val_loss_moe_step_8100.pt
  - last_epoch_moe_0.pt
  - best_val_loss_moe_step_4200.pt
  - last_epoch_moe_1.pt
  - best_val_loss_moe_step_9600.pt
  - best_val_loss_moe_step_7500.pt

Converting ACTIVATED: best_val_loss_moe_step_6000.pt -> moe-activated-converted/best_val_loss_moe_step_6000.bin
✅ Token embeddings: torch.Size([50257, 768])
✅ Output weights: torch.Size([50257, 768])
⚠️  Skipping output.bias (standard GPT-2 doesn't use lm_head bias)
Found expert weights for 5 layers
✅ Converted MoE ACTIVATED checkpoint: moe-activated/best_val_loss_moe_step_6000.pt -> moe-activated-converted/best_val_loss_moe_step_6000.bin
✅ ACTIVATED conversion successful

Converting ACTIVATED: moe_step_9000.pt -> moe-activated-converted/moe_step_9000.bin
✅ To

In [37]:
# Save GPT-2 tokenizer files for MoE model - FIXED VERSION
from transformers import GPT2Tokenizer
import json
import os

print("Saving GPT-2 tokenizer files for MoE model...")

# Load the GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

print(f"Standard GPT-2 tokenizer vocab size: {len(tokenizer)}")
print(f"✅ MoE model vocab size: 50257 (matches actual trained model)")

# Save all tokenizer files to output directory
tokenizer.save_pretrained("./moe-total-converted/")

# Save special_tokens_map.json - simplified approach without problematic `with`
special_tokens_map = {
    "bos_token": "<|endoftext|>",
    "eos_token": "<|endoftext|>",
    "unk_token": "<|endoftext|>",
    "pad_token": "<|endoftext|>"
}

# Simple file writing approach
import json
json_file_path = "moe-total-converted/special_tokens_map.json"
json_data = json.dumps(special_tokens_map, indent=2)

# Write the file simply
f = open(json_file_path, 'w', encoding='utf-8')
f.write(json_data)
f.close()

print("✅ Tokenizer files saved:")
print("  - vocab.json")
print("  - merges.txt") 
print("  - tokenizer_config.json")
print("  - special_tokens_map.json")
print("  - config.json")

print(f"\n✅ Vocab size: {len(tokenizer)}")
print("   Configuration matches actual MoE training")

print("\n🎉 MoE model conversion setup complete!")
print("✅ All files ready for HuggingFace upload!")


Saving GPT-2 tokenizer files for MoE model...
Standard GPT-2 tokenizer vocab size: 50257
✅ MoE model vocab size: 50257 (matches actual trained model)
✅ Tokenizer files saved:
  - vocab.json
  - merges.txt
  - tokenizer_config.json
  - special_tokens_map.json
  - config.json

✅ Vocab size: 50257
   Configuration matches actual MoE training

🎉 MoE model conversion setup complete!
✅ All files ready for HuggingFace upload!


In [16]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GPT2Config
import torch
from huggingface_hub import hf_hub_download

def load_moe_model(checkpoint_name="best_val_loss_moe_step_9000.bin", model_id="idhant297/moe-5l-active-arxiv_code_simplestories"):
    """
    Load a MoE model from HuggingFace Hub with a specific checkpoint.
    
    Args:
        checkpoint_name (str): The checkpoint filename to load
        model_id (str): The HuggingFace model repository ID
    
    Returns:
        tuple: (model, tokenizer) loaded from the checkpoint
    """
    print(f"Loading MoE model from {model_id} checkpoint {checkpoint_name}...")
    
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    
    config = GPT2Config.from_pretrained(model_id)
    
    model = AutoModelForCausalLM.from_config(config)
    
    checkpoint_path = hf_hub_download(
        repo_id=model_id,
        filename=checkpoint_name
    )
    
    state_dict = torch.load(checkpoint_path, map_location="cpu")
    model.load_state_dict(state_dict)
    model.eval()
    
    print(f"✅ MoE model loaded successfully from checkpoint {checkpoint_name}")
    return model, tokenizer

def generate_text_moe(model, tokenizer, prompt, max_length=100, temperature=0.8, top_p=0.95, num_return_sequences=1):
    """
    Generate text using the loaded MoE model.
    
    Args:
        model: The loaded MoE model
        tokenizer: The loaded tokenizer
        prompt (str): Input text prompt
        max_length (int): Maximum length of generated text
        temperature (float): Sampling temperature
        top_p (float): Top-p sampling parameter
        num_return_sequences (int): Number of sequences to generate
    
    Returns:
        list: Generated text sequences
    """
    inputs = tokenizer(prompt, return_tensors="pt")
    
    with torch.no_grad():
        outputs = model.generate(
            inputs["input_ids"],
            max_length=max_length,
            temperature=temperature,
            top_p=top_p,
            do_sample=True,
            num_return_sequences=num_return_sequences,
            pad_token_id=tokenizer.eos_token_id
        )
    
    generated_texts = []
    for output in outputs:
        text = tokenizer.decode(output, skip_special_tokens=True)
        generated_texts.append(text)
    
    return generated_texts

# Example usage for MoE model
checkpoint_name = "best_val_loss_moe_step_9600.bin"
model, tokenizer = load_moe_model(checkpoint_name)

prompt = "test test test"

generated = generate_text_moe(model, tokenizer, prompt, max_length=50)
print(generated)


Loading MoE model from idhant297/moe-5l-active-arxiv_code_simplestories checkpoint best_val_loss_moe_step_9600.bin...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


tokenizer_config.json:   0%|          | 0.00/545 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/130 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/511 [00:00<?, ?B/s]

best_val_loss_moe_step_9600.bin:   0%|          | 0.00/406M [00:00<?, ?B/s]

✅ MoE model loaded successfully from checkpoint best_val_loss_moe_step_9600.bin
['test test test cargo navigating navig danced fateExpl love discovered foes instantly future familiarity fractured enemies fractured emotionsExplExpl companions survivors photos reconnect reconnect journeys embraced embraced partners artifacts partners married embraced bonded emotions emotions bonded partners artifacts experience love companions history intertwined emotions humanity collaborate film passions']
